# The Environmental footprint of data centers in the United States
Script for estimating water footprint, carbon footprint, and water scarcity footprint of data centers 

In [1]:
#import necessary libraries
import warnings; warnings.simplefilter('ignore')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Data centers location and energy use
Ganeshalingam et al. [4] reports likely locations of in-house small and midsize data centers (DC). Detailed information on colocation and hyperscale data centers is derived from commercial compilations [19–21]. Floor space based electricity use are then matched with 2018 estimate of servers by data center type [3]. Scaled server estimates are then spatially distributed to the power control authorities (PCAs) in proportion to the current spatial distribution of installed server bases. PCA boundaries are derived from the Homeland Infrastructure Foundation level data (HIFLD) [34].

In [2]:
PCA_Energy_Use = pd.read_excel(r"\XLS_SI\Input data.xlsx", "Table 1", skiprows = 3, nrows= 72, usecols = 'A,E')
PCA_Energy_Use.head()

,PCA Name,Scaled Power Consumption (MWh)
0,"Alcoa Power Generating, Inc. - Yadkin Division",1485.172374
1,Anchorage Municipal Light & Power,34519.008414
2,Arizona Public Service Company,204051.677389
3,"Arlington Valley, LLC - AVBA",24221.584384
4,"Associated Electric Cooperative, Inc.",46211.323604


# Electricity generation, water consumption, and GHG emission of power plants
Power plant-specific electricity generation and water consumption data come from the US Energy
Information Administration (EIA). We assigned national average values of water consumption per unit
of electricity generation by fuel type (i.e., water intensity; m3/MWh) to all power plants with
unspecified water consumption. Operational water footprints of solar and wind power were taken
from Macknick et al. [25]. Following Grubert [26], we assign all reservoir evaporation to the dam’s
primary purpose (e.g., hydropower). We connected hydroelectric dams with their respective power
plants using data from Grubert [27]. Reservoir specific evaporation comes from Reitz et al. [28]. The U.S. Environmental Protection Agency’s eGRID database [29] provided GHG emissions associated with each power plant.

In [3]:
powerplant = pd.read_excel(r"\XLS_SI\Input data.xlsx", "Table 2", skiprows = 2)
powerplant.head()

,Plant state,Plant name,Plant Id,Balancing Authority Name,Balancing Authority Code,PCA Generation (MWh),Latitude,Longitude,Plant primary fuel,Plant primary fuel code,Net generation (MWh),CO2-e Emission (tons),Water intensity (m3/MWh),Carbon intensity (Tons/MWh),Generation Ratio of Power plant,Water Consumption (m3),Subbasin,HUC8,HUC8 ID
0,AL,ABC Coke,56076,"Southern Company Services, Inc. - Trans",SOCO,2.554145e+08,33.582793,-86.779866,COG,COAL,5290.00,1552.988,1.847267,0.293571,0.000021,9772.043741,Locust,3160111,3160111
1,AL,Alabama Pine Pulp,54429,"Southern Company Services, Inc. - Trans",SOCO,2.554145e+08,31.582500,-87.488900,BLQ,BIOMASS,413079.77,10436.106,1.937552,0.025264,0.001617,800363.491644,Lower Alabama,3150204,3150204
2,AL,Alabama River Pulp,10216,"Southern Company Services, Inc. - Trans",SOCO,2.554145e+08,31.582500,-87.488900,BLQ,BIOMASS,308063.28,12986.077,1.937552,0.042154,0.001206,596888.592313,Lower Alabama,3150204,3150204
3,AL,AMEA Sylacauga Plant,56018,"Southern Company Services, Inc. - Trans",SOCO,2.554145e+08,33.166100,-86.282500,NG,GAS,34570.00,22786.699,0.797026,0.659147,0.000135,27553.194096,Lower Coosa,3150107,3150107
4,AL,ANAD Solar Array,60680,"Southern Company Services, Inc. - Trans",SOCO,2.554145e+08,33.626728,-85.969481,SUN,SOLAR,17168.00,0.000,0.007600,0.000000,0.000067,130.476800,Middle Coosa,3150106,3150106


# Inter PCA electricity transfers
Within the electrical grid, electricity supply matches electricity demand by balancing electricity generation within and transferred into/out of a power control area (PCA). Annual inter-PCA electricity transfers reported by the Federal Energy Regulatory Commission [36] are represented within this approach. Following the approach used by Colett et al. [32], if a PCA is net exporter of electrcity (i.e. total export - total import > 0), internal electricity is assumed to support all the utlities within that PCA. If a PCA is net importer of electricity, deficiency (i.e total import - total export) is assumed to be supplied from the interconnected PCAs once the internal generation is fully utilized.




In [4]:
PCA_balance = pd.read_excel(r"\XLS_SI\Input data.xlsx", "Table 3", skiprows = 2)
PCA_balance.head()

,Responded PCA ID,Responded PCA Name,Reporting Year,Connecting PCA,Connecting PCA ID,Electricity Generation (MWh),Import ratio,Water intensity (m3/MWH)
0,140.0,City of Tallahassee,2018,Duke Energy Florida Inc,234,2848324.999,0.0,0.850009
1,140.0,City of Tallahassee,2018,Tampa Electric Company,262,2848324.999,0.0,1.063299
2,140.0,City of Tallahassee,2018,Florida Power & Light Company,171,2848324.999,0.0,1.132117
3,140.0,City of Tallahassee,2018,Florida Municipal Power Pool,170,2848324.999,0.0,1.358868
4,140.0,City of Tallahassee,2018,"Southern Company Services, Inc. - Trans",253,2848324.999,0.0,7.470213


# Estimating electricity supply to data centers by PCA after accounting for inter PCA electricity transfers

If a PCA is net exporter of electrcity, data center (as well as water and wastewater utilities) draws on electricity produced within its PCA.  In case of net importer, electricity imports from other PCAs are utilized.

In [5]:
#Generate a cross table
table = pd.pivot_table(PCA_Energy_Use, values='Scaled Power Consumption (MWh)', index=['PCA Name'], columns=['PCA Name'], aggfunc=np.sum)
table.fillna(0, inplace= True)
flattened = pd.DataFrame(table.to_records())

#Connect electricity sypply table with inter PCA electricity transfers data
Electricity_supply_profile = PCA_balance.merge(flattened, left_on = "Responded PCA Name", right_on= "PCA Name", how = "left")
Electricity_supply_profile.fillna(0, inplace = True)
PCA = Electricity_supply_profile["Connecting PCA"]
Electricity_supply_profile.tail()

#Calculate the self supplied and imported amount of electricity for each PCA
Balanced_Electricity_supply_profile = Electricity_supply_profile.iloc[:,12:].multiply(Electricity_supply_profile["Import ratio"], axis="index")
Balanced_Electricity_supply_profile1 = pd.concat([PCA, Balanced_Electricity_supply_profile], axis = 1)
Balanced_Electricity_supply_profile2= Balanced_Electricity_supply_profile1.groupby(["Connecting PCA"]).sum().reset_index()
Balanced_Electricity_supply_profile2.head()


,Connecting PCA,"Arlington Valley, LLC - AVBA","Associated Electric Cooperative, Inc.",Avangrid Renewables LLC,Avista Corporation,Balancing Authority of Northern California,Bonneville Power Administration,California Independent System Operator,Chugach Electric Assn Inc,City of Homestead,...,"Southern Company Services, Inc. - Trans",Southwest Power Pool,Southwestern Power Administration,Tampa Electric Company,Tennessee Valley Authority,Tucson Electric Power Company,Turlock Irrigation District,Western Area Power Administration - Desert Southwest Region,Western Area Power Administration - Rocky Mountain Region,Western Area Power Administration UGP West
0,Alberta Electric System Operator,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
1,"Alcoa Power Generating, Inc. - Yadkin Division",0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
2,Anchorage Municipal Light & Power,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
3,Arizona Public Service Company,0.000000,0.0,0.0,0.0,0.0,0.0,379171.751728,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,8019.882716,0.0,45128.117583,0.0,0.0
4,"Arlington Valley, LLC - AVBA",24221.584384,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0


# Indirect water footprint of data centers
Electricity supplied from each power plant is estimated as the fraction of electricity generated by a power plant compared to its operating PCA. Indirect water footprint associated with electricity used during the operation of a data center from power plant is finally aggreagated at HUC8 level.

In [7]:
#Connect the power plants with balnced PCA electricity supply
powerplant_balanced_PCA = powerplant.merge(Balanced_Electricity_supply_profile2, left_on = "Balancing Authority Name", right_on ="Connecting PCA",how = "left")
powerplant_balanced_PCA.fillna(0, inplace = True)
HUC8_Identifier = powerplant_balanced_PCA[["HUC8 ID", "Subbasin"]]
HUC8_Identifier.tail()

,HUC8 ID,Subbasin
8398,10180009,Middle North Platte-Scotts Bluff
8399,10120201,Upper Belle Fourche
8400,10120201,Upper Belle Fourche
8401,10120201,Upper Belle Fourche
8402,10120201,Upper Belle Fourche


In [9]:
#Estimate the indirect water supplied (IWS) from each power plant
IWS_from_PP = powerplant_balanced_PCA.iloc[:,20:].multiply(powerplant_balanced_PCA["Generation Ratio of Power plant"], axis="index").multiply(powerplant_balanced_PCA["Water intensity (m3/MWh)"], axis="index")

#Add the HUC8 identifier of each power plant
PP_with_HUC8 = pd.concat([HUC8_Identifier, IWS_from_PP], axis = 1)

#Aggregate the indirect water footprint at HUC8 level by each PCA
IWF_HUC8_by_PCA= PP_with_HUC8.groupby(["HUC8 ID", "Subbasin"]).sum().reset_index()

#Aggregate the total indirect water footprint at HUC8 level
IWF_HUC8_by_PCA["HUC8 ID"] = IWF_HUC8_by_PCA["HUC8 ID"].astype(str)
IWF_HUC8_by_PCA["Indirect water footprint (m3)"] = IWF_HUC8_by_PCA.sum(axis=1, numeric_only=True)
IWF_HUC8 = IWF_HUC8_by_PCA[["HUC8 ID", "Subbasin", "Indirect water footprint (m3)"]]
IWF_HUC8.head()

,HUC8 ID,Subbasin,Indirect water footprint (m3)
0,1010001,Upper St. John,13805.431499
1,1010004,Aroostook,70766.651048
2,1010005,Meduxnekeag,42.941563
3,1020001,West Branch Penobscot,8127.934015
4,1020003,Mattawamkeag,42.911341


# Direct water footprint of data centers

Direct water consumption of a data center can be estimated from the heat generation capacity
of a data center [42], which is related to the amount of electricity used [43]. Estimates of data center
specific electricity demand were multiplied by the typical water cooling requirement [1] – 1.8
m3/MWh – to estimate the direct water footprint of each data center. The direct water consumption
is assigned to the watershed where the water utility supplying the data center withdraws its water using ArcGIS.

In [85]:
DWF_at_HUC8 = pd.read_excel(r"\XLS_SI\Input data.xlsx", "Table 5", skiprows = 2, usecols = 'A,B,C')
DWF_at_HUC8.head()

,HUC8 ID,Subbasin,Scaled Power Consumption (MWh)
0,1010001,Upper St. John,887.732534
1,1010002,Allagash,0.000000
2,1010003,Fish,1242.825547
3,1010004,Aroostook,1953.011574
4,1010005,Meduxnekeag,1597.918561


In [86]:
#Estimate the direct water supply from each subbasin
DWF_at_HUC8["Direct water footprint (m3)"] = HUC8_DWF["Scaled Power Consumption (MWh)"]*1.8
DWF_at_HUC8.head()

,HUC8 ID,Subbasin,Scaled Power Consumption (MWh),Direct water footprint (m3)
0,1010001,Upper St. John,887.732534,1597.918561
1,1010002,Allagash,0.000000,0.000000
2,1010003,Fish,1242.825547,2237.085985
3,1010004,Aroostook,1953.011574,3515.420833
4,1010005,Meduxnekeag,1597.918561,2876.253409


# Total water footprint of data centers
Total water footprint is the sum of direct water consumption, and indirect water consumption associated with electricity used by data centers, public water system (PWS), and wastewater treatement plants (WWTP) that service a data center. Indirect water use by PWS and WWTP can be estimated using the similar approach mentioned above for indirect water supply from power plants to data centers. This script only accounts for the water and carbon footprint associated with the electricity use at the data center facility. 

In [89]:
#Blue water footprint (BWF) is the sum of direct and indirect water consumption
DWF_at_HUC8["HUC8 ID"] = DWF_at_HUC8["HUC8 ID"].astype(str)
BWF_DC =DWF_at_HUC8.merge(IWF_HUC8[["HUC8 ID", "Indirect water footprint (m3)"]], left_on = [ "HUC8 ID"], right_on = ["HUC8 ID"], how = "left")
BWF_DC.fillna(value={"Indirect water footprint (m3)":0}, inplace = True)
BWF_DC["Blue water footprint (m3)"] = BWF_DC["Direct water footprint (m3)"]+BWF_DC["Indirect water footprint (m3)"]
BWF_DC.head()

,HUC8 ID,Subbasin,Scaled Power Consumption (MWh),Direct water footprint (m3),Indirect water footprint (m3),Blue water footprint (m3)
0,1010001,Upper St. John,887.732534,1597.918561,13805.431499,15403.350060
1,1010002,Allagash,0.000000,0.000000,0.000000,0.000000
2,1010003,Fish,1242.825547,2237.085985,0.000000,2237.085985
3,1010004,Aroostook,1953.011574,3515.420833,70766.651048,74282.071882
4,1010005,Meduxnekeag,1597.918561,2876.253409,42.941563,2919.194972


In [ ]:
#Export the output to excel
BWF_DC.to_excel("BWF_DataCenter.xlsx")

# Carbon footprint of data centers

Electricity supplied from each power plant is estimated as the fraction of electricity generated by a power plant compared to its operating PCA. Carbon footprint associated with electricity used during the operation of a data center from power plant is finally aggreagated at HUC8 level.


In [90]:
#Estimate the GHG emission (GHGI) from each power plant
GHGI_from_PP = powerplant_balanced_PCA.iloc[:,20:].multiply(powerplant_balanced_PCA["Generation Ratio of Power plant"], axis="index").multiply(powerplant_balanced_PCA["Carbon intensity (Tons/MWh)"], axis="index")

#Add the HUC8 identifier of each power plant
GHGI_with_HUC8 = pd.concat([HUC8_Identifier, GHGI_from_PP], axis = 1)

#Aggregate the emission at HUC8 level by each PCA
GHGI_HUC8_by_PCA= GHGI_with_HUC8.groupby(["HUC8 ID", "Subbasin"]).sum().reset_index()

#Aggregate the carbon footprtin (CF) at HUC8 level
GHGI_HUC8_by_PCA["HUC8 ID"] = GHGI_HUC8_by_PCA["HUC8 ID"].astype(str)
GHGI_HUC8_by_PCA["Carbon Footprint (Tons CO2-eq)"] = GHGI_HUC8_by_PCA.sum(axis=1, numeric_only=True)
CF_HUC8 = GHGI_HUC8_by_PCA[["HUC8 ID", "Subbasin", "Carbon Footprint (Tons CO2-eq)"]]
CF_HUC8.head()

,HUC8 ID,Subbasin,Carbon Footprint (Tons CO2-eq)
0,1010001,Upper St. John,0.000000
1,1010004,Aroostook,1195.299195
2,1010005,Meduxnekeag,0.000000
3,1020001,West Branch Penobscot,0.000000
4,1020003,Mattawamkeag,0.000000


In [28]:
#Export the output to excel
CF_DC.to_excel("CF_DataCenter.xlsx")

# Water scarcity footprint of data centers
We quantified the W SF of data centers using the AWARE method set forth by Boulay et al. [46] (see the Supportive Information for more details). Other societal and environmental water use data, as well as data on natural water availability within each US watershed, for estimating the characterization factors of subbasins come from ref [47–49].


In [91]:
Characterization_factors = pd.read_excel(r"\XLS_SI\Input data.xlsx", "Table 5", skiprows = 2, usecols = 'A,B,D')
Characterization_factors.tail()

,HUC8 ID,Subbasin,Characterization factor
2094,18100100,Southern Mojave,100.000000
2095,18100201,Whitewater River,20.279640
2096,18100202,Carrizo Creek,46.577097
2097,18100203,San Felipe Creek,31.673298
2098,18100204,Salton Sea,100.000000


In [97]:
#Multiply the water consumption from each subbasin by its characterization factor to get the water scarcity footprint of data centers
Characterization_factors["HUC8 ID"] = Characterization_factors['HUC8 ID'].astype(str)
HUC_WSF = BWF_DC.merge(Characterization_factors, left_on = ["HUC8 ID", "Subbasin"], right_on = ["HUC8 ID", "Subbasin"], how = "left")
HUC_WSF["Water Scarcity Footprint (m3-eq)"] = HUC_WSF["Blue water footprint (m3)"]*HUC_WSF["Characterization factor"]
DC_portfolio_at_HUC8 = HUC_WSF[["HUC8 ID", "Subbasin", "Scaled Power Consumption (MWh)", "Direct water footprint (m3)", "Indirect water footprint (m3)", "Blue water footprint (m3)", "Characterization factor", "Water Scarcity Footprint (m3-eq)"]]
DC_portfolio_at_HUC8.head(5)

,HUC8 ID,Subbasin,Scaled Power Consumption (MWh),Direct water footprint (m3),Indirect water footprint (m3),Blue water footprint (m3),Characterization factor,Water Scarcity Footprint (m3-eq)
0,1010001,Upper St. John,887.732534,1597.918561,13805.431499,15403.350060,0.240074,3697.939131
1,1010002,Allagash,0.000000,0.000000,0.000000,0.000000,0.334706,0.000000
2,1010003,Fish,1242.825547,2237.085985,0.000000,2237.085985,0.311171,696.117157
3,1010004,Aroostook,1953.011574,3515.420833,70766.651048,74282.071882,0.309205,22968.408809
4,1010005,Meduxnekeag,1597.918561,2876.253409,42.941563,2919.194972,0.277446,809.918126


In [20]:
DC_portfolio_at_HUC8.to_excel("DC_portfolio_at_HUC8.xlsx")

# Notes

The authors of this iPython notebook make this code available with the MIT license, 2021. 
 https://opensource.org/licenses/MIT
 
All the description of methodology and references mentioned in this scripts can be found in the main manuscript.

Siddik, M. A. B., Shehabi, A., & Marston, L. T. (2021). The environmental footprint of data centers in the United States. Environmental Research Letters. https://doi.org/10.1088/1748-9326/abfba1
